In [ ]:
'''
An investment strategy that selects the 50 stocks with the
highest price momentum
'''

In [ ]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

In [ ]:
stocks = pd.read_csv('stocks.csv')
from apitoken import IEX_CLOUD_API_TOKEN

In [ ]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [ ]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'
            ],
            index = my_columns),
            ignore_index = True
        ) 

final_dataframe

In [ ]:
#remove low momentum stocks, 50 highest momentum stocks, highest momentum at the top
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True, drop = True)

In [ ]:
#calculating the number of shares to buy
while True:
    try:
        portfolio_size = float(input('Enter the value of your portfolio in £: '))
        if portfolio_size > 0:
            break
        print('Please Enter a valid number: ')
    except ValueError:
        print('Please Enter a valid number: ')

In [ ]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = round(position_size/final_dataframe.loc[i, 'Price'], 5)

final_dataframe

In [ ]:
#more realistic momentum strategy, with high and low quality momentum stocks
hqm_columns = ['Ticker', 'Price', 'Number of Shares to Buy',
              'One Year Price Return', 'One Year Return Percentile',
              'Six Month Price Return', 'Six Month Return Percentile',
              'Three Month Price Return', 'Three Month Return Percentile',
              'One Month Price Return', 'One Month Return Percentile',
              'HQM Score']

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [symbol, data[symbol]['price'], 'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],
            index = hqm_columns),
            ignore_index = True
        )

In [ ]:
hqm_dataframe = hqm_dataframe.dropna()

In [ ]:
time_periods = ['One Year',
               'Six Month',
               'Three Month',
               'One Month']

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

hqm_dataframe

In [ ]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

In [ ]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True, drop = True)
hqm_dataframe

In [ ]:
#calculating the number of shares to buy
while True:
    try:
        portfolio_size = float(input('Enter the value of your portfolio in £: '))
        if portfolio_size > 0:
            break
        print('Please Enter a valid number: ')
    except ValueError:
        print('Please Enter a valid number: ')

In [ ]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = round(position_size/hqm_dataframe.loc[i, 'Price'], 5)

hqm_dataframe

In [ ]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format({
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
})

pound_format = writer.book.add_format({
        'num_format': '£0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
})

integer_format = writer.book.add_format({
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
})

percent_format = writer.book.add_format({
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
})

In [ ]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price', pound_format],
    'C': ['Number of Shares to Buy', integer_format],
    'D': ['One Year Price Return', percent_format],
    'E': ['One Year Return Percentile', percent_format],
    'F': ['Six Month Price Return', percent_format],
    'G': ['Six Month Return Percentile', percent_format],
    'H': ['Three Month Price Return', percent_format],
    'I': ['Three Month Return Percentile', percent_format],
    'J': ['One Month Price Return', percent_format],
    'K': ['One Month Return Percentile', percent_format],
    'L': ['HQM Score', percent_format]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 18, string_format)

writer.save()